# Algorithm Investigations

This notebook is a play area to examine various algorithms for choosing the next map choices.

## Algorithm factors

### Map Group

This is an arbitrary categorisation of maps to enable configuration of weighting attributes at a group level.

Each map group has a `weight` (0-100) and a `repeat_decay` (0.0 - 1.0).

**Weight** indicates a base probability of selection, the higher the number the more likely that a map is chosen.

**Repeat factor** is a multiplier that affects the weighting on other instances of the same map. For example, if Carentan Day is
selected, the repeat factor is applied to the weighting of other Carentan maps e.g. Night to make it less likely for the
same map to be selected multiple times. A repeat factor of 0.0 will prevent the map from being selected multiple times.

### Environments

As with map groups, this is an arbitrary categorisation of environments (e.g. day, rain, night) to enable configuration
of weighting attributes at a group level.

**Repeat factor** works the same as for the map group. We prefer day maps to other environments, so we allow day to
repeat without limit (`repeat_decay` = 1.0) but we prevent other environments from being selected more than once by
setting `repeat_decay` = 0.0.

```json
{
  "groups": {
      "Boost": {
          "weight": 80,
          "repeat_decay": 0.6,
          "maps": [
              "carentan",
              "omahabeach",
              "stmariedumont",
              "stmereeglise",
              "utahbeach"
          ]
      },
      "Normal": {
          "weight": 50,
          "repeat_decay": 0.25,
          "maps": [
              "elsenbornridge",
              "foy",
              "hill400",
              "hurtgenforest",
              "kharkov",
              "kursk",
              "mortain",
              "purpleheartlane",
              "stalingrad"
          ]
      },
      "Unboost": {
          "weight": 30,
          "repeat_decay": 0,
          "maps": ["driel", "elalamein", "remagen"]
      }
  },
  "environments": {
      "Boost": {
          "repeat_decay": 1.0,
          "environments": ["day"]
      },
      "Normal": {
          "repeat_decay": 0.1,
          "environments": [
              "rain",
              "overcast"
          ]
      },
      "Unboost": {
          "repeat_decay": 0.0,
          "environments": [
              "dusk",
              "night",
              "dawn"
          ]
      }
  }
}
```

When exploded, the above map group configuration would look like this:

| map             | map_group | map_weight | map_repeat_decay |
| --------------- | --------- | ---------- | ----------------- |
| carentan        | Boost     | 80         | 0.6               |
| driel           | Unboost   | 30         | 0.0               |
| elalamein       | Unboost   | 30         | 0.0               |
| elsenbornridge  | Normal    | 50         | 0.25              |
| foy             | Normal    | 50         | 0.25              |
| hill400         | Normal    | 50         | 0.25              |
| hurtgenforest   | Normal    | 50         | 0.25              |
| kharkov         | Normal    | 50         | 0.25              |
| kursk           | Normal    | 50         | 0.25              |
| mortain         | Normal    | 50         | 0.25              |
| omahabeach      | Boost     | 80         | 0.6               |
| purpleheartlane | Normal    | 50         | 0.25              |
| remagen         | Unboost   | 30         | 0.0               |
| stalingrad      | Normal    | 50         | 0.25              |
| stmariedumont   | Boost     | 80         | 0.6               |
| stmereeglise    | Boost     | 80         | 0.6               |
| utahbeach       | Boost     | 80         | 0.6               |

The exploded environment configuration would look like this:

| environment | environment_group | environment_repeat_decay |
| ----------- | ----------------- | ------------------------- |
| day         | Boost             | 1.0                       |
| dusk        | Unboost           | 0.0                       |
| rain        | Unboost           | 0.0                       |
| night       | Unboost           | 0.0                       |
| dawn        | Unboost           | 0.0                       |
| overcast    | Unboost           | 0.0                       |



## Imports

In [1]:
import numpy as np
import pandas as pd
import json
from pathlib import Path


## Read config

In this section we read the weighting configuration and parse it into DataFrames.

In [3]:
config = {
    "groups": {
        "Boost": {
            "weight": 80,
            "repeat_decay": 0.6,
            "maps": [
                "carentan",
                "omahabeach",
                "stmariedumont",
                "stmereeglise",
                "utahbeach",
            ],
        },
        "Normal": {
            "weight": 50,
            "repeat_decay": 0.25,
            "maps": [
                "elsenbornridge",
                "foy",
                "hill400",
                "hurtgenforest",
                "kharkov",
                "kursk",
                "mortain",
                "purpleheartlane",
                "stalingrad",
            ],
        },
        "Unboost": {
            "weight": 30,
            "repeat_decay": 0,
            "maps": ["driel", "elalamein", "remagen"],
        },
    },
    "environments": {
        "Boost": {
            "weight": 100,
            "repeat_decay": 0.9,
            "environments": ["day"],
        },
        "Normal": {
            "weight": 90,
            "repeat_decay": 0.5,
            "environments": [
                "rain",
                "overcast",
                "dusk",
                "dawn",
            ],
        },
        "Unboost": {
            "weight": 50,
            "repeat_decay": 0.0,
            "environments": [
                "night",
            ],
        },
    },
}

df_map_groups = (
    pd.DataFrame.from_dict(config["groups"], orient="index")
    .reset_index(names="group")
    .explode("maps")
).rename(
    columns={
        "maps": "map",
        "weight": "map_weight",
        "group": "map_group",
        "repeat_decay": "map_repeat_decay",
    }
).set_index("map")

df_environments = (
    pd.DataFrame.from_dict(config["environments"], orient="index")
    .reset_index(names="environment")
    .explode("environments")
).rename(
    columns={
        "environment": "environment_category",
        "environments": "environment",
        "weight": "environment_weight",
        "repeat_decay": "environment_repeat_decay",
    }
).set_index("environment")
# df_environments
# df_map_groups

## Read map definitions

In this section we read the JSON map definitions (these come from the CRCON server) and marshal them into a DataFrame.

In [4]:
# Load map definitions as read from the server
contents = Path("get_maps.json").read_text()
data = json.loads(contents)

df_maps = pd.json_normalize(
    data["result"],
    None,
    meta=["id", "environment", "game_mode", ["map", "id", "name", "pretty_name"]],
)

# Convert certain columns to categories
cols = ['game_mode', 'environment', 'map.id']
df_maps[cols] = df_maps[cols].astype("category")

sorted_df = df_maps.sort_values(by=["map.id", "environment"])
df_warfare = df_maps.loc[df_maps["game_mode"] == "warfare"]

# Calculate the number of instances of each map.id
map_instance_counts = df_warfare.groupby(["map.id"], observed=True).size()
map_instance_counts.name = "map_instance_count"

environment_instance_counts = df_warfare.groupby(["environment"], observed=True).size()
environment_instance_counts.name = "environment_instance_count"



## Select map layers

In this section we select the layers that will be offered in the votemap. Layers are selected iteratively as follows:

1. Calculate an overall weighting score for each layer. This is derived from the base `map_weight` multiplied by various
   scores to generate an overall score.

2. Marshal the layers into two similar arrays (layer ID and overall weighting) that we use to make a weighted choice.

3. Call `np.random.choice` to select one layer.

4. Based on the attributes of the selected layer, modify the weighting scores of similar layers to reduce their
   probability of selection next time.

5. Repeat until we have selected the required number of layers.


In [30]:
# Create a dataframe that joins the maps to the configuration parameters
from pickletools import dis


df = (
    df_warfare.join(
        map_instance_counts,
        on="map.id",
    ).join(
        environment_instance_counts,
        on="environment",
    )
    .join(
        df_map_groups,
        on="map.id",
    )
    .join(df_environments, on="environment")
)

current_layer_id = "carentan_warfare"
recent_layer_history = []
number_last_played_to_exclude = 0
current_side = None
consider_offensive_same_map = False
consider_skirmishes_as_same_map = False


standard_exclusions = {current_layer_id} | set(
            recent_layer_history[: number_last_played_to_exclude]
        )
df=df[~df["id"].isin(standard_exclusions)]


if consider_offensive_same_map or consider_skirmishes_as_same_map:
    maps_to_exclude = {
        df.loc[df["id"] == layer.id]["map.id"].values[0]
        for layer in recent_layer_history[: number_last_played_to_exclude]
    }
    df = df[~df["map.id"].isin(maps_to_exclude)]
    
# Validate that all maps are configured - if there are any maps on the server that we haven't had configured then
# all we can do is log and drop them
nan_rows = df.loc[df['map_group'].isnull()]
if len(nan_rows):
    display(nan_rows["map.id"].unique().tolist())
# Validate that all environments are configured
nan_rows = df.loc[df['environment_category'].isnull()]
if len(nan_rows):
    display(nan_rows["environment"].tolist())
df = df.dropna(subset=["map_group", "environment_category"])

# The normalization factors address the fact that some map_ids have more instances (layers) than others, and the same
# for environments. Without normalization, this would make it more likely to select a map with more layers, or an
# environment with more instances, which would skew the selection.
df["map_count_normalization_factor"] = 1 / df["map_instance_count"]
df["environment_count_normalization_factor"] = 1 / df["environment_instance_count"]

# Initialize the repeat scores to 1.0, which has no effect on the weightings
df["map_repeat_score"] = 1.0
df["environment_repeat_score"] = 1.0

selected_layers: list[str] = []

for i in range(6):
    # (Re)calculate overall weightings. The factors won't have changed (these come from configuration) but the scores
    # are changed in response to the map that has been selected.
    df["overall_weighting"] = (
        df["map_weight"]
        * df["map_count_normalization_factor"]
        * df["environment_weight"]
        * df["environment_count_normalization_factor"]
        * df["map_repeat_score"]
        * df["environment_repeat_score"]
    )

    # Create two arrays of equal length, one with the layer IDs and one with the corresponding weighting
    maps_array = df["id"].to_numpy()
    weights_array = df["overall_weighting"].to_numpy()

    # Select a layer for inclusion in the votemap choices
    selected_layer = np.random.choice(
        maps_array, 1, replace=True, p=(weights_array / sum(weights_array))
    )[0]
    selected_layers.append(selected_layer)

    # Based on the attributes of the selected layer, modify the weightings of similar layers to reduce their probability
    # of selection
    selected_layer_props = df.loc[df["id"] == selected_layer]
    selected_map_id = selected_layer_props["map.id"].values[0]
    selected_environment = selected_layer_props["environment"].values[0]
    selected_environment_category = selected_layer_props["environment_category"].values[0]
    # display(selected_environment)

    # Modify weightings based on what has just been chosen
    df.loc[df["map.id"] == selected_map_id, "map_repeat_score"] = (
        df.map_repeat_score * df.map_repeat_decay
    )
    df.loc[df["environment_category"] == selected_environment_category, "environment_repeat_score"] = (
        df.environment_repeat_score * df.environment_repeat_decay
    )
    # Prevent the chosen layer from being chosen again
    df.loc[df["id"] == selected_layer, "map_repeat_score"] = 0.0


# Create a small DataFrame with the ordered results of the selection
df_results = df[["id", "pretty_name", "map.id", "environment"]].set_index("id")
df_selected = pd.DataFrame(selected_layers, columns=["layer_id"]).join(
    df_results, on="layer_id",
)
df_selected


{'carentan_warfare'}

12     False
13     False
25      True
26     False
29     False
30     False
33     False
34     False
44     False
45     False
46     False
49     False
50     False
53     False
56     False
57     False
60     False
61     False
64     False
65     False
73     False
75     False
76     False
77     False
80     False
81     False
84     False
85     False
88     False
89     False
92     False
93     False
96     False
97     False
100    False
101    False
Name: id, dtype: bool

,id,game_mode,attackers,environment,pretty_name,image_name,map.id,map.name,map.tag,map.pretty_name,...,map.axis.team,map.orientation,map_instance_count,environment_instance_count,map_group,map_weight,map_repeat_decay,environment_category,environment_weight,environment_repeat_decay
12,PHL_L_1944_Warfare,warfare,None,rain,Purple Heart Lane Warfare (Rain),purpleheartlane-rain.webp,purpleheartlane,PURPLE HEART LANE,PHL,Purple Heart Lane,...,axis,vertical,2,1,Normal,50,0.25,Normal,90,0.5
13,PHL_L_1944_Warfare_Night,warfare,None,night,Purple Heart Lane Warfare (Night),purpleheartlane-night.webp,purpleheartlane,PURPLE HEART LANE,PHL,Purple Heart Lane,...,axis,vertical,2,13,Normal,50,0.25,Unboost,50,0.0
26,carentan_warfare_night,warfare,None,night,Carentan Warfare (Night),carentan-night.webp,carentan,CARENTAN,CAR,Carentan,...,axis,horizontal,2,13,Boost,80,0.60,Unboost,50,0.0
29,driel_warfare,warfare,None,day,Driel Warfare,driel-day.webp,driel,DRIEL,DRL,Driel,...,axis,vertical,2,16,Unboost,30,0.00,Boost,100,0.9
30,driel_warfare_night,warfare,None,night,Driel Warfare (Night),driel-night.webp,driel,DRIEL,DRL,Driel,...,axis,vertical,2,13,Unboost,30,0.00,Unboost,50,0.0
33,elalamein_warfare,warfare,None,day,El Alamein Warfare,elalamein-day.webp,elalamein,EL ALAMEIN,ELA,El Alamein,...,axis,horizontal,2,16,Unboost,30,0.00,Boost,100,0.9
34,elalamein_warfare_night,warfare,None,dusk,El Alamein Warfare (Dusk),elalamein-dusk.webp,elalamein,EL ALAMEIN,ELA,El Alamein,...,axis,horizontal,2,4,Unboost,30,0.00,Normal,90,0.5
44,elsenbornridge_warfare_day,warfare,None,day,Elsenborn Ridge Warfare,elsenbornridge-day.webp,elsenbornridge,ELSENBORN RIDGE,EBR,Elsenborn Ridge,...,axis,vertical,3,16,Normal,50,0.25,Boost,100,0.9
45,elsenbornridge_warfare_morning,warfare,None,dawn,Elsenborn Ridge Warfare (Dawn),elsenbornridge-dawn.webp,elsenbornridge,ELSENBORN RIDGE,EBR,Elsenborn Ridge,...,axis,vertical,3,1,Normal,50,0.25,Normal,90,0.5
46,elsenbornridge_warfare_night,warfare,None,night,Elsenborn Ridge Warfare (Night),elsenbornridge-night.webp,elsenbornridge,ELSENBORN RIDGE,EBR,Elsenborn Ridge,...,axis,vertical,3,13,Normal,50,0.25,Unboost,50,0.0


,layer_id,pretty_name,map.id,environment
0,mortain_skirmish_dusk,Mortain Warfare (Dusk),mortain,dusk
1,omahabeach_warfare_night,Omaha Beach Warfare (Dusk),omahabeach,dusk
2,kursk_warfare,Kursk Warfare,kursk,day
3,PHL_L_1944_Warfare_Night,Purple Heart Lane Warfare (Night),purpleheartlane,night
4,elsenbornridge_warfare_day,Elsenborn Ridge Warfare,elsenbornridge,day
5,foy_warfare,Foy Warfare,foy,day


In [26]:
df

,id,game_mode,attackers,environment,pretty_name,image_name,map.id,map.name,map.tag,map.pretty_name,...,map_weight,map_repeat_decay,environment_category,environment_weight,environment_repeat_decay,map_count_normalization_factor,environment_count_normalization_factor,map_repeat_score,environment_repeat_score,overall_weighting
12,PHL_L_1944_Warfare,warfare,None,rain,Purple Heart Lane Warfare (Rain),purpleheartlane-rain.webp,purpleheartlane,PURPLE HEART LANE,PHL,Purple Heart Lane,...,50,0.25,Normal,90,0.5,0.500000,1.000000,0.00,0.125,0.00000
13,PHL_L_1944_Warfare_Night,warfare,None,night,Purple Heart Lane Warfare (Night),purpleheartlane-night.webp,purpleheartlane,PURPLE HEART LANE,PHL,Purple Heart Lane,...,50,0.25,Unboost,50,0.0,0.500000,0.076923,0.25,0.000,0.00000
25,carentan_warfare,warfare,None,day,Carentan Warfare,carentan-day.webp,carentan,CARENTAN,CAR,Carentan,...,80,0.60,Boost,100,0.9,0.500000,0.062500,0.00,0.810,0.00000
26,carentan_warfare_night,warfare,None,night,Carentan Warfare (Night),carentan-night.webp,carentan,CARENTAN,CAR,Carentan,...,80,0.60,Unboost,50,0.0,0.500000,0.076923,0.60,0.000,0.00000
29,driel_warfare,warfare,None,day,Driel Warfare,driel-day.webp,driel,DRIEL,DRL,Driel,...,30,0.00,Boost,100,0.9,0.500000,0.062500,1.00,0.810,84.37500
30,driel_warfare_night,warfare,None,night,Driel Warfare (Night),driel-night.webp,driel,DRIEL,DRL,Driel,...,30,0.00,Unboost,50,0.0,0.500000,0.076923,1.00,0.000,0.00000
33,elalamein_warfare,warfare,None,day,El Alamein Warfare,elalamein-day.webp,elalamein,EL ALAMEIN,ELA,El Alamein,...,30,0.00,Boost,100,0.9,0.500000,0.062500,1.00,0.810,84.37500
34,elalamein_warfare_night,warfare,None,dusk,El Alamein Warfare (Dusk),elalamein-dusk.webp,elalamein,EL ALAMEIN,ELA,El Alamein,...,30,0.00,Normal,90,0.5,0.500000,0.250000,1.00,0.125,42.18750
44,elsenbornridge_warfare_day,warfare,None,day,Elsenborn Ridge Warfare,elsenbornridge-day.webp,elsenbornridge,ELSENBORN RIDGE,EBR,Elsenborn Ridge,...,50,0.25,Boost,100,0.9,0.333333,0.062500,0.25,0.810,23.43750
45,elsenbornridge_warfare_morning,warfare,None,dawn,Elsenborn Ridge Warfare (Dawn),elsenbornridge-dawn.webp,elsenbornridge,ELSENBORN RIDGE,EBR,Elsenborn Ridge,...,50,0.25,Normal,90,0.5,0.333333,1.000000,0.00,0.125,0.00000
